In [4]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, models, transforms

# Transfer learning on the CIFAR-10 dataset

In [13]:
# Hyperparameters
learning_rate = 1e-4
batch_size = 50
NUM_EPOCHS = 4  # alter this afterwards
momentum = 0.9
loss_function = nn.CrossEntropyLoss()

# Architecture
NUM_CLASSES = 10


In [14]:
RESIZE_SIZE = 70
CROP_TO_SIZE = 64

transform = transforms.Compose([
    transforms.Resize((RESIZE_SIZE, RESIZE_SIZE)),
    transforms.RandomCrop((CROP_TO_SIZE, CROP_TO_SIZE)),
    transforms.ToTensor(),
])


cifar_10_training_data = datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
cifar_10_test_data = datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(
    cifar_10_training_data, batch_size=batch_size, shuffle=True, num_workers=2
)
test_loader = DataLoader(
    cifar_10_test_data, batch_size=batch_size, shuffle=False, num_workers=2
)

Files already downloaded and verified
Files already downloaded and verified


Training-Testing Functions

In [15]:
writer = SummaryWriter()

In [16]:
BATCH_TO_PRINT = 100

In [17]:
def train_model(data_loader, network, optimizer, criterion, num_epochs=NUM_EPOCHS):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    network.to(device)
    network.train()
    for epoch in range(num_epochs):
        running_loss = epoch_loss = 0.
        for i, data in enumerate(data_loader):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = network(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            epoch_loss += loss.item()

            if i % BATCH_TO_PRINT == (BATCH_TO_PRINT - 1):
                print('[Epoch %d, Batch %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / BATCH_TO_PRINT))
                running_loss = 0.0

        writer.add_scalar(f'Loss/train:', epoch_loss / len(data_loader), epoch)
        print(f"[{epoch + 1}] loss: {epoch_loss / len(data_loader):.3f}")

    writer.flush()
    return network


def test_model(data_loader, network):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    network.to(device)
    network.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = network(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(F'Test accuracy: {(100 * correct / total):.2f}%')

## Fine-tuning the model

In [7]:
alexnet = models.alexnet(weights='AlexNet_Weights.DEFAULT')
alexnet.classifier.add_module('6', nn.Linear(4096, NUM_CLASSES))

optimizer = optim.SGD(alexnet.parameters(), lr=learning_rate, momentum=momentum)

In [8]:
trained_network = train_model(train_loader, alexnet, optimizer, loss_function, num_epochs=2)
object_to_save = {
    'model_state_dict': trained_network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}
torch.save(object_to_save, 'alexnet-fine-tuned-cifar-10.pt')

[Epoch 1, Batch   100] loss: 1.863
[Epoch 1, Batch   200] loss: 1.439
[Epoch 1, Batch   300] loss: 1.345
[Epoch 1, Batch   400] loss: 1.252
[Epoch 1, Batch   500] loss: 1.187
[Epoch 1, Batch   600] loss: 1.163
[Epoch 1, Batch   700] loss: 1.158
[Epoch 1, Batch   800] loss: 1.113
[Epoch 1, Batch   900] loss: 1.120
[Epoch 1, Batch  1000] loss: 1.050
[1] loss: 1.269
[Epoch 2, Batch   100] loss: 1.011
[Epoch 2, Batch   200] loss: 1.030
[Epoch 2, Batch   300] loss: 1.005
[Epoch 2, Batch   400] loss: 1.004
[Epoch 2, Batch   500] loss: 0.983
[Epoch 2, Batch   600] loss: 0.987
[Epoch 2, Batch   700] loss: 0.978
[Epoch 2, Batch   800] loss: 0.944
[Epoch 2, Batch   900] loss: 0.936
[Epoch 2, Batch  1000] loss: 0.923
[2] loss: 0.980


In [ ]:
trained_network = train_model(train_loader, alexnet, optimizer, loss_function, num_epochs=2)
object_to_save = {
    'model_state_dict': trained_network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}
torch.save(object_to_save, 'alexnet-fine-tuned-cifar-10.pt')

In [9]:
test = test_model(test_loader, trained_network)

Test accuracy: 67.99%


## Feature extraction

In [10]:
alexnet = models.alexnet(weights='AlexNet_Weights.DEFAULT')

# Freeze all the layers in the feature extractor
for param in alexnet.features.parameters():
    param.requires_grad = False

alexnet.classifier.add_module('6', nn.Linear(4096, NUM_CLASSES))

optimizer = optim.SGD(alexnet.parameters(), lr=learning_rate, momentum=momentum)
trained_network = train_model(train_loader, alexnet, optimizer, loss_function)
object_to_save = {
    'model_state_dict': trained_network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}
torch.save(object_to_save, 'alexnet-feature-extracted-cifar-10.pt')

[Epoch 1, Batch   100] loss: 1.846
[Epoch 1, Batch   200] loss: 1.530
[Epoch 1, Batch   300] loss: 1.431
[Epoch 1, Batch   400] loss: 1.446
[Epoch 1, Batch   500] loss: 1.391
[Epoch 1, Batch   600] loss: 1.372
[Epoch 1, Batch   700] loss: 1.337
[Epoch 1, Batch   800] loss: 1.314
[Epoch 1, Batch   900] loss: 1.333
[Epoch 1, Batch  1000] loss: 1.348
[1] loss: 1.435
[Epoch 2, Batch   100] loss: 1.312
[Epoch 2, Batch   200] loss: 1.274
[Epoch 2, Batch   300] loss: 1.273
[Epoch 2, Batch   400] loss: 1.283
[Epoch 2, Batch   500] loss: 1.312
[Epoch 2, Batch   600] loss: 1.264
[Epoch 2, Batch   700] loss: 1.272
[Epoch 2, Batch   800] loss: 1.261
[Epoch 2, Batch   900] loss: 1.265
[Epoch 2, Batch  1000] loss: 1.282
[2] loss: 1.280
[Epoch 3, Batch   100] loss: 1.247
[Epoch 3, Batch   200] loss: 1.250
[Epoch 3, Batch   300] loss: 1.264
[Epoch 3, Batch   400] loss: 1.239
[Epoch 3, Batch   500] loss: 1.241
[Epoch 3, Batch   600] loss: 1.224
[Epoch 3, Batch   700] loss: 1.188
[Epoch 3, Batch   800] 

In [11]:
test_model(test_loader, trained_network)

Test accuracy: 58.98%


# Transfer learning from MNIST to SVHN

In [2]:
import torch
import torch.nn as nn

MNIST_IMAGE_SIZE = 28

num_input_channels = 1
num_output_classes = 10

num_conv1_channels = 32
conv_kernel_size = 3
conv_stride = 1
conv_padding = 1
pool_kernel_size = 2
num_conv2_channels = 64

fc1_output_size = 128
fc2_output_size = 64
fc3_output_size = 32

dropout_rate = 0.25

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=num_input_channels,
            out_channels=num_conv1_channels,
            kernel_size=conv_kernel_size,
            stride=conv_stride,
            padding=conv_padding,
        )
        self.conv2 = nn.Conv2d(
            in_channels=num_conv1_channels,
            out_channels=num_conv2_channels,
            kernel_size=conv_kernel_size,
            stride=conv_stride,
            padding=conv_padding,
        )
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=pool_kernel_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(
            num_conv2_channels * MNIST_IMAGE_SIZE**2 // pool_kernel_size**4,
            fc1_output_size,
        )
        self.fc2 = nn.Linear(fc1_output_size, fc2_output_size)
        self.fc3 = nn.Linear(fc2_output_size, fc3_output_size)
        self.fc4 = nn.Linear(fc3_output_size, num_output_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        return x



Set up the dataset and data loader

In [23]:
MNIST_MEAN, MNIST_STD = (0.1307,), (0.3081,)
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(MNIST_MEAN, MNIST_STD)]
)
mnist_training_data = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
mnist_test_data = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=mnist_training_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test_data, batch_size=64, shuffle=False)

# Instantiate the model and set up the optimizer and loss function

In [31]:
learning_rate = 1e-3

my_model = CNN()
optimizer = optim.Adam(my_model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [32]:
trained_network = train_model(train_loader, my_model, optimizer, loss_function, num_epochs=4)

[Epoch 1, Batch   100] loss: 1.335
[Epoch 1, Batch   200] loss: 0.449
[Epoch 1, Batch   300] loss: 0.287
[Epoch 1, Batch   400] loss: 0.221
[Epoch 1, Batch   500] loss: 0.189
[Epoch 1, Batch   600] loss: 0.181
[Epoch 1, Batch   700] loss: 0.157
[Epoch 1, Batch   800] loss: 0.136
[Epoch 1, Batch   900] loss: 0.128
[1] loss: 0.334
[Epoch 2, Batch   100] loss: 0.101
[Epoch 2, Batch   200] loss: 0.116
[Epoch 2, Batch   300] loss: 0.107
[Epoch 2, Batch   400] loss: 0.099
[Epoch 2, Batch   500] loss: 0.098
[Epoch 2, Batch   600] loss: 0.095
[Epoch 2, Batch   700] loss: 0.094
[Epoch 2, Batch   800] loss: 0.099
[Epoch 2, Batch   900] loss: 0.099
[2] loss: 0.101
[Epoch 3, Batch   100] loss: 0.086
[Epoch 3, Batch   200] loss: 0.084
[Epoch 3, Batch   300] loss: 0.066
[Epoch 3, Batch   400] loss: 0.083
[Epoch 3, Batch   500] loss: 0.065
[Epoch 3, Batch   600] loss: 0.081
[Epoch 3, Batch   700] loss: 0.073
[Epoch 3, Batch   800] loss: 0.075
[Epoch 3, Batch   900] loss: 0.069
[3] loss: 0.076
[Epoch 

In [33]:
torch.save(trained_network.state_dict(), 'my-cnn-mnist.pt')

In [34]:
test_model(test_loader, trained_network)

Test accuracy: 99.06%


## Use pre-trained model for SVNH dataset

In [39]:
pretrained_model = CNN()
pretrained_model.load_state_dict(torch.load('my-cnn-mnist.pt'))

# Freeze the weights of the pretrained model
for param in pretrained_model.parameters():
    param.requires_grad = False

Modify the last layer of the model to output 10 classes instead of 2

In [40]:
pretrained_model.fc1.requires_grad = True

# Modify the first convolutional layer to accept 3 channels instead of 1
pretrained_model.conv1 = nn.Conv2d(
    in_channels=3,
    out_channels=num_conv1_channels,
    kernel_size=conv_kernel_size,
    stride=conv_stride,
    padding=conv_padding,
)
SVHN_IMAGE_SIZE = 32
pretrained_model.fc1 = nn.Linear(
    num_conv2_channels * SVHN_IMAGE_SIZE**2 // pool_kernel_size**4,
    fc1_output_size,
)

Set up the dataset and data loader

In [19]:
transform = transforms.Compose([
    transforms.Resize((SVHN_IMAGE_SIZE, SVHN_IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
svhn_train_data = datasets.SVHN(root='./data', split='train', transform=transform, download=True)
svhn_test_data = datasets.SVHN(root='./data', split='test', transform=transform, download=True)
svhn_train_loader = torch.utils.data.DataLoader(
    dataset=svhn_train_data, batch_size=128, shuffle=True
)
svhn_test_loader = torch.utils.data.DataLoader(
    dataset=svhn_test_data, batch_size=128, shuffle=False
)

Using downloaded and verified file: ./data\train_32x32.mat
Using downloaded and verified file: ./data\test_32x32.mat


Set up the optimizer and loss function

In [42]:
optimizer = optim.Adam(pretrained_model.fc2.parameters(), lr=learning_rate)

In [43]:
trained_network = train_model(
    svhn_train_loader, pretrained_model, optimizer, loss_function, num_epochs=4
)
torch.save(trained_network.state_dict(), 'my-cnn-mnist-pretrained-svhn.pt')

[Epoch 1, Batch   100] loss: 2.319
[Epoch 1, Batch   200] loss: 2.320
[Epoch 1, Batch   300] loss: 2.321
[Epoch 1, Batch   400] loss: 2.321
[Epoch 1, Batch   500] loss: 2.316
[1] loss: 2.320
[Epoch 2, Batch   100] loss: 2.317
[Epoch 2, Batch   200] loss: 2.321
[Epoch 2, Batch   300] loss: 2.320
[Epoch 2, Batch   400] loss: 2.319
[Epoch 2, Batch   500] loss: 2.319
[2] loss: 2.319
[Epoch 3, Batch   100] loss: 2.316
[Epoch 3, Batch   200] loss: 2.319
[Epoch 3, Batch   300] loss: 2.321
[Epoch 3, Batch   400] loss: 2.322
[Epoch 3, Batch   500] loss: 2.323
[3] loss: 2.320
[Epoch 4, Batch   100] loss: 2.321
[Epoch 4, Batch   200] loss: 2.319
[Epoch 4, Batch   300] loss: 2.317
[Epoch 4, Batch   400] loss: 2.320
[Epoch 4, Batch   500] loss: 2.322
[4] loss: 2.319


In [44]:
test_model(svhn_test_loader, trained_network)

Test accuracy: 6.38%


## Transfer learning

In [20]:
pretrained_model = CNN()
pretrained_model.load_state_dict(torch.load('my-cnn-mnist.pt'))

for param in pretrained_model.parameters():
    param.requires_grad = True
    

pretrained_model.conv1 = nn.Conv2d(
    in_channels=3,
    out_channels=num_conv1_channels,
    kernel_size=conv_kernel_size,
    stride=conv_stride,
    padding=conv_padding,
)
SVHN_IMAGE_SIZE = 32
pretrained_model.fc1 = nn.Linear(
    num_conv2_channels * SVHN_IMAGE_SIZE**2 // pool_kernel_size**4,
    fc1_output_size,
)
optimizer = optim.SGD(pretrained_model.parameters(), lr=0.001, momentum=0.9)
trained_network = train_model(
    svhn_train_loader, pretrained_model, optimizer, loss_function, num_epochs=10
)
torch.save(trained_network.state_dict(), 'my-cnn-mnist-transfer-svhn.pt')

[Epoch 1, Batch   100] loss: 2.288
[Epoch 1, Batch   200] loss: 2.262
[Epoch 1, Batch   300] loss: 2.246
[Epoch 1, Batch   400] loss: 2.236
[Epoch 1, Batch   500] loss: 2.231
[1] loss: 2.249
[Epoch 2, Batch   100] loss: 2.209
[Epoch 2, Batch   200] loss: 2.170
[Epoch 2, Batch   300] loss: 2.125
[Epoch 2, Batch   400] loss: 2.070
[Epoch 2, Batch   500] loss: 1.983
[2] loss: 2.085
[Epoch 3, Batch   100] loss: 1.814
[Epoch 3, Batch   200] loss: 1.716
[Epoch 3, Batch   300] loss: 1.610
[Epoch 3, Batch   400] loss: 1.519
[Epoch 3, Batch   500] loss: 1.444
[3] loss: 1.591
[Epoch 4, Batch   100] loss: 1.301
[Epoch 4, Batch   200] loss: 1.255
[Epoch 4, Batch   300] loss: 1.190
[Epoch 4, Batch   400] loss: 1.136
[Epoch 4, Batch   500] loss: 1.084
[4] loss: 1.175
[Epoch 5, Batch   100] loss: 1.011
[Epoch 5, Batch   200] loss: 0.978
[Epoch 5, Batch   300] loss: 0.962
[Epoch 5, Batch   400] loss: 0.915
[Epoch 5, Batch   500] loss: 0.917
[5] loss: 0.947
[Epoch 6, Batch   100] loss: 0.850
[Epoch 6, 

In [21]:
test_model(svhn_test_loader, trained_network)

Test accuracy: 84.36%
